# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

In [1]:
import multiprocessing
import re

## Step 1: Use the requests library to retrieve the content from the URL below.

In [2]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [3]:
r = requests.get(url).content
#r

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [4]:
from bs4 import BeautifulSoup

In [5]:
html_content = r
soup = BeautifulSoup(html_content, 'lxml')
#links = soup.find_all('a')
links = [a.get('href') for a in soup.find_all('a', href=True)]  
links = list(set(links)) #unique velues of the list
#links

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [6]:
domain = 'http://wikipedia.org'

In [7]:
pattern = '^(http|https):'
absolute_links = [x for x in links if re.search(pattern, x)]
#absolute_links

In [8]:
pattern = '^(\/)wiki'
relative_links = [x for x in links if '%' not in x  if re.search(pattern, x)]
#relative_links

In [9]:
links = [i+j for i,j in zip(absolute_links,relative_links)]
links = list(set(links)) #unique velues of the list

#links

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [10]:
import os

In [11]:
# create a folder named 'wikipedia' at cwd:
try:
    os.makedirs('wikipedia')
except FileExistsError:
    print("The folder '/wikipedia' already exists.")

The folder '/wikipedia' already exists.


In [12]:
cwd = os.getcwd()
cwd

'/Users/victorgalan/ironhack/git/dataptmad1019/module-1/lab-parallelization/your-code'

In [13]:
try:
    os.replace(cwd+'/main.ipynb',cwd+'/wikipedia/main.ipynb')
except FileNotFoundError:
    print('No se encuentra el archivo.')

In [14]:
cwd

'/Users/victorgalan/ironhack/git/dataptmad1019/module-1/lab-parallelization/your-code'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [15]:
from slugify import slugify

In [16]:
def index_page(link):
    r = requests.get(link).text
    slugify(link)
    file = open('./wikipedia/slugified', 'w')
    file.write(r)
    file.close()

In [17]:
index_page(links[1])

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [18]:
%%time
for link in links:
    index_page(link)

CPU times: user 3 s, sys: 209 ms, total: 3.2 s
Wall time: 2min 9s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [19]:
import multiprocessing

In [21]:
print("Number of cpu: ", multiprocessing.cpu_count())

Number of cpu :  8


In [22]:
%%timeit
pool = multiprocessing.Pool()
md_files = pool.map(index_page, links)
pool.terminate()
pool.join()

14.1 s ± 633 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
